In [1]:
# create a hdf5 dataset out of goetiff files

In [1]:
import numpy as np
import pandas as pd
import h5py

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

from glob import glob
#from os import listdir
#from os.path import splitext
import logging
import os

In [2]:
# directory paths
#dir_img = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/256x256/1m/data_256_ortho/'
#dir_mask = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/256x256/1m/data_256_gt_roads/'
dir_img = '/home/philipp/Data/edin_dataset/'

In [5]:
sufix = 'ortho/'
ids_ortho = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
sufix = 'dsm/'
ids_dsm = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
sufix = 'dtm/'
ids_dtm = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
sufix = 'slope/'
ids_slope = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]

In [32]:
sufix = 'ortho'
ids_ortho = []
for i in range(100):
    ids_ortho.append(dir_img + sufix + '/tile_' + sufix + '_' + str(i) + '.tif')
    
sufix = 'dsm'
ids_dsm = []
for i in range(100):
    ids_dsm.append(dir_img + sufix + '/tile_' + sufix + '_' + str(i) + '.tif')
    
sufix = 'dtm'
ids_dtm = []
for i in range(100):
    ids_dtm.append(dir_img + sufix + '/tile_' + sufix + '_' + str(i) + '.tif')
    
sufix = 'slope'
ids_slope = []
for i in range(100):
    ids_slope.append(dir_img + sufix + '/tile_' + sufix + '_' + str(i) + '.tif')

In [22]:
print(ids_ortho[:4])
print(ids_dsm[:4])
print(ids_dtm[:4])
print(ids_slope[:4])

['/home/philipp/Data/edin_dataset/ortho/tile_ortho_0.tif', '/home/philipp/Data/edin_dataset/ortho/tile_ortho_1.tif', '/home/philipp/Data/edin_dataset/ortho/tile_ortho_2.tif', '/home/philipp/Data/edin_dataset/ortho/tile_ortho_3.tif']
['tile_dsm_52', 'tile_dsm_33', 'tile_dsm_84', 'tile_dsm_46']
['tile_dtm_17', 'tile_dtm_15', 'tile_dtm_11', 'tile_dtm_18']
['tile_slope_72', 'tile_slope_85', 'tile_slope_44', 'tile_slope_35']


In [38]:
from osgeo import gdalconst

def tif2array(input_file, dtype=np.uint8):
    """
    read GeoTiff and convert to numpy.ndarray.
    Inputs:
        input_file (str) : the name of input GeoTiff file.
    return:
        image(np.array) : image for each bands
        dataset : for gdal's data drive.
    """
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    # Allocate our array using the first band's datatype
    image_datatype = dataset.GetRasterBand(1).DataType
    print(image_datatype)
    print(dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount)
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                     dtype=dtype)
    
    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        image[:, :, b] = band.ReadAsArray()#buf_type=gdalconst.GDT_Byte)
        
    #image = image[2:-2,2:-2,:]
        
    return image

In [39]:
# convert image to numpy array
img = tif2array(ids_ortho[10], np.uint8)
print(ids_ortho[10])

1
515 515 4
/home/philipp/Data/edin_dataset/ortho/tile_ortho_10.tif


In [40]:
img.shape

(515, 515, 4)

In [41]:
img

array([[[ 76,  83,  76, 164],
        [ 78,  87,  79, 167],
        [ 62,  64,  53, 150],
        ...,
        [ 42,  46,  46, 111],
        [ 40,  43,  43, 106],
        [ 40,  41,  42, 106]],

       [[ 70,  76,  68, 155],
        [ 81,  91,  84, 169],
        [ 62,  65,  55, 149],
        ...,
        [ 42,  46,  46, 111],
        [ 44,  46,  48, 115],
        [ 42,  44,  46, 114]],

       [[ 60,  64,  57, 139],
        [ 69,  76,  68, 156],
        [ 61,  64,  55, 145],
        ...,
        [ 41,  44,  45, 108],
        [ 46,  50,  51, 122],
        [ 44,  46,  47, 116]],

       ...,

       [[ 86,  73,  53, 183],
        [ 70,  57,  42, 171],
        [ 86,  77,  56, 190],
        ...,
        [ 82,  81,  58, 187],
        [ 94,  97,  70, 197],
        [108, 115,  89, 209]],

       [[ 95,  86,  61, 192],
        [ 93,  84,  61, 192],
        [ 98,  92,  68, 198],
        ...,
        [ 73,  70,  49, 180],
        [106, 114,  91, 209],
        [114, 126, 105, 216]],

       [[126

In [42]:
img.dtype

dtype('uint8')

In [73]:
# convert image to numpy array
img_ortho = tif2array(ids_ortho[10], np.uint8)
print(ids_ortho[10])
print(img_ortho.shape)
print(img_ortho.dtype)

# convert image to numpy array
img_dsm = tif2array(ids_dsm[15], np.float32)
print(ids_dsm[15])
print(img_dsm.shape)
print(img_dsm.dtype)

# convert image to numpy array
img_dtm = tif2array(ids_dtm[15], np.float32)
print(ids_dtm[15])
print(img_dtm.shape)
print(img_dtm.dtype)

# convert image to numpy array
img_slope = tif2array(ids_slope[15], np.float32)
print(ids_slope[15])
print(img_slope.shape)
print(img_slope.dtype)

1
515 515 4
/home/philipp/Data/edin_dataset/ortho/tile_ortho_10.tif
(515, 515, 4)
uint8
6
516 516 1
/home/philipp/Data/edin_dataset/dsm/tile_dsm_15.tif
(516, 516, 1)
float32
7
516 516 1
/home/philipp/Data/edin_dataset/dtm/tile_dtm_15.tif
(516, 516, 1)
float32
6
516 516 1
/home/philipp/Data/edin_dataset/slope/tile_slope_15.tif
(516, 516, 1)
float32


In [74]:
img_ortho = img_ortho[1:-2,1:-2]
img_dsm = img_dsm[2:-2,2:-2]
img_dtm = img_dtm[2:-2,2:-2]
img_slope = img_slope[2:-2,2:-2]

In [78]:
print(img_ortho.shape)
print(img_dsm.shape)
print(img_dtm.shape)
print(img_slope.shape)

(512, 512, 4)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)


In [ ]:
# Data Standardization
# y = (x - mean) / standard_deviation
# mean = sum(x) / count(x)
# standard_deviation = sqrt( sum( (x - mean)^2 ) / count(x))

In [ ]:
# Data Normalization
# y = (x - min) / (max - min)

In [94]:
means = []
stds = []

for img in [img_ortho, img_dsm, img_dtm, img_slope]:
    means.append(np.mean(img, dtype='float32'))
    stds.append(np.std(img, dtype='float32'))
print(means)
print(stds)

[106.623276, 14.312894, 1325.6674, 29.272326]
[54.006954, 12.487458, 16.26564, 7.691262]


In [95]:
nor_img_ortho = (img_ortho - means[0]) / stds[0]
nor_img_dsm = (img_dsm - means[1]) / stds[1]
nor_img_dtm = (img_dtm - means[2]) / stds[2]
nor_img_slope = (img_slope - means[3]) / stds[3]

In [98]:
print(nor_img_ortho.shape)
print(nor_img_ortho.dtype)

(512, 512, 4)
float32


In [99]:
import matplotlib

ModuleNotFoundError: No module named 'matplotlib'

In [34]:
temp = []
for i in ids:
    temp.append(tif2array(dir_mask + ids[0] + ".tif"))

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

...

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

In [35]:
ar_ground_truth = np.array(temp)

In [38]:
temp=0

In [12]:
# hdf5

In [36]:
# initialising hdf5
ds_file_name  = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/256x256/1m/dataset_256.hdf5";
file = h5py.File(ds_file_name, "w");

In [37]:
# Create a dataset in the file

ds_ortho = file.create_dataset("ortho", np.shape(ar_ortho), h5py.h5t.STD_U8BE, data=ar_ortho)

ds_ground_truth = file.create_dataset("ground_truth", np.shape(ar_ground_truth), h5py.h5t.STD_U8BE, data=ar_ground_truth)

file.close()